In [1]:
import SimpleITK as sitk

import numpy as np
import os
from ipywidgets import interact, fixed

import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(precision=4, suppress=True)

In [2]:
data_dir = "E:\\Guided Research\AMOS_examples"
# data_dir = "E:\\Guided Research\AMOS22"
raw_dir = data_dir + "\imagesTr\\"
labels_dir = data_dir + "\labelsTr\\"

target_dir = "E:\\Guided Research\AMOS_preprocessed"
target_raw_dir = target_dir + "\imagesTr\\"
target_labels_dir = target_dir + "\labelsTr\\"
# list(os.listdir(raw_dir))

In [3]:
list(os.listdir(raw_dir))

['amos_0001.nii.gz',
 'amos_0004.nii.gz',
 'amos_0005.nii.gz',
 'amos_0006.nii.gz',
 'amos_0007.nii.gz',
 'amos_0009.nii.gz',
 'amos_0010.nii.gz',
 'amos_0011.nii.gz']

In [4]:
img_list = list(os.listdir(raw_dir))
label_list = list(os.listdir(labels_dir))

In [5]:
print(len(img_list))
print(len(label_list))
# img_list.__len__() == label_list.__len__()


8
8


In [6]:
img1 = sitk.ReadImage(raw_dir+img_list[0])
label1 = sitk.ReadImage(labels_dir+label_list[0])

In [7]:
print(sitk.GetArrayFromImage(img1).shape)
print(sitk.GetArrayFromImage(label1).shape)

(90, 768, 768)
(90, 768, 768)


In [8]:
# from scipy.signal import resample_poly
# from scipy.ndimage import zoom
# from torch.nn.functional import interpolate
# import torch
# def resampling_method(img):
#     voxel_sizes = img.GetSpacing()
#     target_spacing  = (2,2,5)
#     inverse_ratios = (target_spacing[0]/voxel_sizes[0],target_spacing[1]/voxel_sizes[1],target_spacing[2]/voxel_sizes[2])
#     print(inverse_ratios)
    
#     img_array = sitk.GetArrayFromImage(img)
#     img_array = np.expand_dims(img_array,axis=(0,1))
#     # print(img_array.shape)
#     img_array = torch.tensor(img_array)
#     print(img_array.shape)
#     resized_img_array = interpolate(img_array,scale_factor=inverse_ratios).squeeze()
#     resized_img = sitk.GetImageFromArray(resized_img_array).CopyInformation(img)
#     resized_img.SetSpacing((2,2,5))
#     print(inverse_ratios)
#     print(resized_img.GetSpacing())
#     return resized_img

In [9]:
def resampling_method(volume, new_spacing, interpolator=sitk.sitkLinear, default_value=0):
    """
    It resamples the original volume to have the voxel size equal to the desired one.
    Parameters
    ----------
    volume: sitk image 
        The original volume
    new_spacing: numpy.array of float (i.e. [1.15, 1.30, 0.75])
        The desired voxel size
    Returns
    ----------
    sitk image
        The input volume resampled with the desired voxel size
    """
    original_size = volume.GetSize()
    original_spacing = volume.GetSpacing()
    new_size = [int((original_size[0] - 1) * original_spacing[0] / new_spacing[0]), 
                    int((original_size[1] - 1) * original_spacing[1] / new_spacing[1]), 
                            int((original_size[2] - 1) * original_spacing[2] / new_spacing[2])]
    # print(new_size)
    new_volume = sitk.Resample(volume, new_size, sitk.Transform(), interpolator, volume.GetOrigin(),
                                new_spacing, volume.GetDirection(), default_value, volume.GetPixelID())
    return new_volume

In [10]:
resampled_img1 = resampling_method(img1,(2,2,5))

[218, 218, 89]


In [100]:
def print_header(img):
    selected_image = img
    print('Before modification:')
    print('origin: ' + str(selected_image.GetOrigin()))
    print('size: ' + str(selected_image.GetSize()))
    print('spacing: ' + str(selected_image.GetSpacing()))
    print('direction: ' + str(selected_image.GetDirection()))
    print('pixel type: ' + str(selected_image.GetPixelIDTypeAsString()))
    print('number of pixel components: ' + str(selected_image.GetNumberOfComponentsPerPixel()))

    # selected_image.SetOrigin((78.0, 76.0, 77.0))
    # selected_image.SetSpacing([0.5,0.5,3.0])

    # print('\nAfter modification:')
    # print('origin: ' + str(selected_image.GetOrigin()))
    # print('spacing: ' + str(selected_image.GetSpacing()))

In [102]:
print_header(img1)

Before modification:
origin: (-233.0, 373.4296875, 26.5)
size: (768, 768, 90)
spacing: (1.0, 1.0, 5.0)
direction: (1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
pixel type: 32-bit float
number of pixel components: 1


In [103]:
print_header(resampled_img1)

Before modification:
origin: (-233.0, 373.4296875, 26.5)
size: (218, 218, 89)
spacing: (2.0, 2.0, 5.0)
direction: (1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
pixel type: 32-bit float
number of pixel components: 1


In [52]:
print_header(label1)

Before modification:
origin: (-233.0, 373.4296875, 26.5)
size: (768, 768, 90)
spacing: (0.5703125, 0.5703125, 5.0)
direction: (1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
pixel type: 16-bit unsigned integer
number of pixel components: 1

After modification:
origin: (78.0, 76.0, 77.0)
spacing: (0.5, 0.5, 3.0)


In [14]:
resampled_label = resampling_method(label1,(2,2,5))

[218, 218, 89]


In [6]:
nib.is_proxy(img.dataobj)

True

In [7]:
img_data = img.get_fdata()

In [8]:
unique,counts = np.unique(img_data,return_counts=True)

In [9]:
unique,counts

(array([0., 1., 3., 4., 5.]),
 array([27758042,  2865131,  1814305,   421663,   957435], dtype=int64))

In [10]:
np.isclose(img_data,2.0).sum()

0

In [11]:
new_data = np.isclose(img_data,2.0)

In [12]:
new_img = nib.Nifti1Image(new_data,img.affine,img.header)

In [13]:
nib.save(new_img,'img_smth.nii.gz')

In [14]:
img_dup = nib.load('../sample_img/labels-021.nii.gz')
img_dup2 = nib.load('img_smth.nii.gz')

In [15]:
img_dup2.get_fdata().dtype

dtype('float64')

In [16]:
np.allclose(img_dup.get_fdata(),img_dup2.get_fdata())

False